In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from PID_functions import *
%matplotlib inline
np.random.seed(1010)

## PHID decomposition on simulated data

We propose the same time series used to check the correctness of our PID function to check also the PHID decomposition proposed by Mediano. In particular, the sum of the PID atoms and the PHID atoms should be the same as it is the TDMI.

In [ ]:
# generate the time series
time=np.linspace(0,10,5000)
data = np.zeros((2,len(time)))
alpha = 0.9
A = np.array([[alpha,alpha],[0,0]])
S = np.array([[1,0],[0,1]])
for i in range(len(time)-1):
    data[:,i+1] = np.dot(A,data[:,i]) + np.random.multivariate_normal(mean=[0,0],cov=S)

In [ ]:
X = data[:,1:]
Y = data[0,:-1]
Z = data[1,:-1]

TDMI_pid = sum(PID(X,Y,Z))

X1 = data[0,:-1]
X2 = data[1,:-1]
Y1 = data[0,1:]
Y2 = data[1,1:]

dict_solution = PHID(X1,X2,Y1,Y2) #dictionary with values of all PHID atoms
values = list(dict_solution.values()) #list with values of all PHID atoms 

TDMI_phid = sum(values)

print('PID  TDMI:', TDMI_pid)
print('PHID TDMI:', TDMI_phid)
print('     TDMI:', I_XY(X,np.vstack((Y,Z))))

In [ ]:
for key in dict_solution.keys():
    print(f'{key}:',dict_solution[key])

## Real data PHID decomposition

At this point the PHID can be applied to the real data available. As in the PID case, we create one matrix for each PHID atom but this time we only show some combinations of them representing quantities of interest.

In [ ]:
data = scipy.io.loadmat("102715.REST1.RL.GLS.ptseries.mat")
X = data['tseries']

In [ ]:
# PID of real data
S, R, U = SRU(X)

In [ ]:
# PHID of real data
phid = PHID_m(X)

An issue occurred in the PID applied to real data is that the WMS is not always positive suggesting that this is not a good quantity for the description of the evolution of information flow.  
From the decomposition proposed by Mediano, though, we can notice that some terms may be overcounted and indeed that is the case for the R_R atom. Mediano proposes a revisited version of the WMS by adding the R_R term. We can check that in this way the result is zero or porsitive almost everywhere.

In [ ]:
WMS = S - R
WMSr = WMS + phid['R_R']

fig, ax = plt.subplots(figsize=(8,8))
cax = ax.matshow(WMSr)
fig.colorbar(cax)
ax.set_title("WMSr")